In [105]:
import torch
import torch.nn as nn

class CustomTransformer(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_heads, num_layers):
        super(CustomTransformer, self).__init__()
        # Define layers for your custom transformer
        self.transformer = nn.TransformerEncoderLayer(d_model=input_dim, nhead=num_heads)
        self.transformer_encoder = nn.TransformerEncoder(self.transformer, num_layers=num_layers)
        self.fc = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.transformer_encoder(x)
        x = self.fc(x)
        x = self.relu(x)
        return x

# Assuming BERT embeddings size is 768
input_dim = 768
hidden_dim = 100
num_heads = 4
num_layers = 2

# Instantiate your custom transformer model
custom_transformer = CustomTransformer(input_dim, hidden_dim, num_heads, num_layers)

# Freeze the model parameters
for param in custom_transformer.parameters():
    param.requires_grad = False





In [111]:
bert_embeddings = Embeddings().get_embeddings(["hi","hey", "julia hates code", "fuck college", "fuck", "penn", "upenn", "vedha", "julia", "vedha loves pie"])[1].last_hidden_state[:, 0]

# Print the same output multiple times
for _ in range(1000):
    # Use the transformer to generate output
    output = custom_transformer(bert_embeddings).cpu().detach().numpy()
output

array([[0.5655032 , 0.0629204 , 0.035607  , 0.02010247, 1.4503956 ,
        0.09055949, 0.        , 0.421587  , 0.        , 0.        ,
        0.        , 0.6744474 , 0.2500276 , 0.14058623, 0.        ,
        0.14104715, 0.20743507, 0.        , 0.6586079 , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.22981495,
        0.52168685, 0.0683948 , 0.1984303 , 0.        , 0.        ,
        1.152183  , 0.33594042, 0.        , 0.09970921, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.21027005,
        0.        , 0.61612564, 0.        , 0.2756435 , 0.        ,
        0.9495958 , 0.06774285, 0.8426675 , 0.        , 0.        ,
        0.        , 0.        , 0.0373522 , 0.        , 0.42545766,
        0.5773406 , 0.08728976, 0.        , 0.53563184, 0.        ,
        0.        , 0.        , 0.01295132, 0.        , 0.52636856,
        0.36273396, 0.4247045 , 0.        , 0.        , 0.01401085,
        0.        , 0.        , 0.39746425, 0.82

In [76]:
# Print requires_grad status of parameters
for name, param in custom_transformer.named_parameters():
    print(name, param.requires_grad)


transformer.self_attn.in_proj_weight False
transformer.self_attn.in_proj_bias False
transformer.self_attn.out_proj.weight False
transformer.self_attn.out_proj.bias False
transformer.linear1.weight False
transformer.linear1.bias False
transformer.linear2.weight False
transformer.linear2.bias False
transformer.norm1.weight False
transformer.norm1.bias False
transformer.norm2.weight False
transformer.norm2.bias False
transformer_encoder.layers.0.self_attn.in_proj_weight False
transformer_encoder.layers.0.self_attn.in_proj_bias False
transformer_encoder.layers.0.self_attn.out_proj.weight False
transformer_encoder.layers.0.self_attn.out_proj.bias False
transformer_encoder.layers.0.linear1.weight False
transformer_encoder.layers.0.linear1.bias False
transformer_encoder.layers.0.linear2.weight False
transformer_encoder.layers.0.linear2.bias False
transformer_encoder.layers.0.norm1.weight False
transformer_encoder.layers.0.norm1.bias False
transformer_encoder.layers.0.norm2.weight False
transf

In [ ]:
embeddings

In [73]:
# Use BERT embeddings as input to your transformer
output = custom_transformer(bert_embeddings)

# Your model will now always output the same result, as the parameters are frozen
embeddings_dataset = output.cpu().detach().numpy()
embeddings_dataset

array([[0.32492712, 0.        , 0.        ],
       [0.68284047, 0.        , 0.23811845],
       [0.5894346 , 1.0451578 , 0.        ],
       [0.25341946, 0.        , 1.0140357 ]], dtype=float32)

In [24]:
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
import faiss

from transformers import AutoTokenizer, AutoModel

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


class Embeddings:
    #CLS is a special classification token and the last hidden state of BERT Embedding
    def cls_pooling(self, model_output):
        return model_output.last_hidden_state[:, 0]

    #BERT tokenizer of input text
    def get_embeddings(self, text_list):
        encoded_input = tokenizer(
            text_list, padding=True, truncation=True, return_tensors="pt"
        )
        encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
        model_output = model(**encoded_input)
        return self.cls_pooling(model_output).cpu().detach().numpy(), model_output
    
    
    #convert dataset into embeddings dataset to run FAISS
    def makeEmbeddings(self,dataset):
        embeddings = []
        for data in dataset:
            embeddings.append(self.get_embeddings(data)[0])
        return np.array(embeddings)
    
    def getQueryEmbedding(self, query):
        return self.get_embeddings([query])

In [68]:
bert_embeddings = Embeddings().get_embeddings(["hi","hey", "julia hates code", "fuck college"])[1].last_hidden_state[:, 0]


In [39]:
bert_embeddings.is_nested

False

In [107]:

    
class Faiss:
    def __init__(self):
        pass

    def faiss(self,xb):
        d = xb[0].size
        M = 32
        index = faiss.IndexHNSWFlat(d, M)            
        index.hnsw.efConstruction = 40         # Setting the value for efConstruction.
        index.hnsw.efSearch = 16               # Setting the value for efSearch.
        index.add(xb)
        return index
    
    def query(self,index,xq,k=3):
        D, I = index.search(xq, k)   
        return D, I

In [108]:
embeddings_dataset = output[:len(output)-1]
xq = output[len(output)-1:]

In [110]:
xb = embeddings_dataset

index = Faiss().faiss(xb)
D,I = Faiss().query(index,xq)
I

array([[5, 3, 0]])

In [90]:
xq

array([[0.        , 0.68749493, 0.67374915],
       [0.        , 0.43349937, 0.75967026]], dtype=float32)

In [94]:
xb

array([[0.        , 0.3699596 , 0.5188751 ],
       [0.        , 0.561179  , 0.        ],
       [0.        , 0.12295088, 0.        ],
       [0.        , 0.68749493, 0.67374915]], dtype=float32)

In [96]:
output

array([[0.        , 0.3699596 , 0.5188751 ],
       [0.        , 0.561179  , 0.        ],
       [0.        , 0.12295088, 0.        ],
       [0.        , 0.68749493, 0.67374915],
       [0.        , 0.43349937, 0.75967026]], dtype=float32)